In [17]:
import sys
path = '/Users/Everwitt/Documents/Pitch-Prediction/pitcherprediction'
sys.path.append(path)

In [19]:
import pandas as pd
import numpy as np

#importing path to obvious_zones allows us to import
from obvious_zones import *

In [20]:
p = pd.read_csv('../combining-data/all_pitches_results.csv')

In [21]:
# For Each Pitch: 6 pitches only
p['pitch_type'].value_counts()

FF    1250368
FT     680398
SL     566488
CH     364690
CU     292785
FC     191334
Name: pitch_type, dtype: int64

In [22]:
# For Each Zone:
# need to run return zone on every pitch in the data set and then remove zones that are not obvious
p['obv_zone'] = ''

In [23]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res,obv_zone
0,0.416,2.963,C,S,FF,2015000001,0,0,0,1,0.0,0.0,0.0,0,strike,
1,-0.191,2.347,S,S,FF,2015000001,0,1,0,2,0.0,0.0,0.0,1,strike,
2,-0.518,3.284,F,S,FF,2015000001,0,2,0,3,0.0,0.0,0.0,1,foul,
3,-0.641,1.221,B,B,FF,2015000001,0,2,0,4,0.0,0.0,0.0,0,ball,
4,-1.821,2.083,B,B,CU,2015000001,1,2,0,5,0.0,0.0,0.0,0,ball,


In [24]:
# https://bit.ly/3j43EqB
# we get this number from (44.08 + 18.29)/2 / 12 = 2.59875
# where 44.08 is the universal top, 18.29 the universal bottom
# 3.673333333, 1.524166667 are top and bottom in ft (2.59875 is the middle)
# 3.673333333 - 2.59875 = 1.074583333
# 1.524166667 - 2.59875 = -1.074583333
rescale_y = 2.599


p['py'] = p['py'] - rescale_y

In [26]:
# These values are actually based on the universal strike zone values
RIGHT = 0.831
TOP = 1.074
obv_zones = ObviousZones(RIGHT, TOP)

In [27]:
#adding obv zone info to the df 
for i, row in p.iterrows():    
    p.at[i, 'obv_zone'] = obv_zones.return_zone(row['px'], row['py'])

In [28]:
#see strikes vs balls
p['obv_zone'].value_counts()

-1     1634414
12b     420117
15b     416363
13b     389123
16b     179081
10b     140269
14b      94646
9b       47673
11b      24377
Name: obv_zone, dtype: int64

In [29]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res,obv_zone
0,0.416,0.364,C,S,FF,2015000001,0,0,0,1,0.0,0.0,0.0,0,strike,-1
1,-0.191,-0.252,S,S,FF,2015000001,0,1,0,2,0.0,0.0,0.0,1,strike,-1
2,-0.518,0.685,F,S,FF,2015000001,0,2,0,3,0.0,0.0,0.0,1,foul,-1
3,-0.641,-1.378,B,B,FF,2015000001,0,2,0,4,0.0,0.0,0.0,0,ball,15b
4,-1.821,-0.516,B,B,CU,2015000001,1,2,0,5,0.0,0.0,0.0,0,ball,12b


In [30]:
#remove strike data
filt = p['obv_zone'] != '-1'
p = p.loc[filt]

In [31]:
print(p.shape)
print(p['obv_zone'].value_counts())

(1711649, 16)
12b    420117
15b    416363
13b    389123
16b    179081
10b    140269
14b     94646
9b      47673
11b     24377
Name: obv_zone, dtype: int64


In [32]:
p.to_csv('./obv_zone_cutoff.csv', index=False)

In [33]:
testdf = pd.read_csv('./obv_zone_cutoff.csv')

In [34]:
testdf.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res,obv_zone
0,-0.641,-1.378,B,B,FF,2015000001,0,2,0,4,0.0,0.0,0.0,0,ball,15b
1,-1.821,-0.516,B,B,CU,2015000001,1,2,0,5,0.0,0.0,0.0,0,ball,12b
2,-1.088,-0.989,B,B,FF,2015000002,0,0,1,1,0.0,0.0,0.0,0,ball,12b
3,1.470,-0.249,B,B,FF,2015000003,0,0,1,1,0.0,1.0,0.0,0,ball,13b
4,-1.337,-0.701,B,B,FF,2015000003,1,0,1,2,0.0,1.0,0.0,0,ball,12b
